In [2]:
import pysam
from collections import defaultdict

In [3]:
# Read vcf
vcf_file = "test1_data_1.annotated.vcf"
vcf = pysam.VariantFile(vcf_file)

In [4]:
## Total variants
total_variants = defaultdict(int)

# Count variants per chromosome
for rec in vcf.fetch():
    total_variants[rec.chrom] += 1

# Total variants
variant_count = sum(total_variants.values())
print(f"Total variants in '{vcf_file}': {variant_count}")

Total variants in 'test1_data_1.annotated.vcf': 30726


In [5]:
## Find column indices for SYMBOL, PHENO and SLIN_SIG
symbol_index = pheno_index = clin_index = None
with open(vcf_file) as f:
    for line in f:
        if line.startswith("##INFO=<ID=CSQ"):
            header = line.split("Format:")[1].split('"')[0].strip()
            csq_fields = header.split("|")
            symbol_index = csq_fields.index("SYMBOL") if "SYMBOL" in csq_fields else None
            pheno_index = csq_fields.index("PHENO") if "PHENO" in csq_fields else None
            clin_index  = csq_fields.index("CLIN_SIG") if "CLIN_SIG" in csq_fields else None
            break

if symbol_index is None or pheno_index is None or clin_index is None:
    raise ValueError("SYMBOL, PHENO, or CLIN_SIG field not found in CSQ header")

In [6]:
## Count unique genes
unique_genes = set()
with open(vcf_file) as f:
    for line in f:
        if line.startswith("#"):
            continue
        info = line.strip().split("\t")[7]
        for entry in info.split(";"):
            if entry.startswith("CSQ="):
                csq_entries = entry[len("CSQ="):].split(",")
                for csq in csq_entries:
                    fields = csq.split("|")
                    if len(fields) > symbol_index:
                        gene = fields[symbol_index]
                        if gene:
                            unique_genes.add(gene)
                break

print(f"Number of unique genes: {len(unique_genes)}")
print("Few genes:", list(unique_genes)[:20])

Number of unique genes: 15890
Few genes: ['PLEKHA1', 'JDP2', 'SMU1', 'ACOT4', 'NKG7', 'ZNF341', 'FAM91A1', 'RBM23', 'AC074117.13', 'MIR98', 'WDR91', 'RP11-344E13.3', 'RNF182', 'SNX19', 'OR4K15', 'PCDHGB7', 'PSMA7', 'NFATC2', 'CTD-2306M10.1', 'MAGEF1']


In [7]:
## Count unique traits/disease conditions
unique_traits = set()
with open(vcf_file) as f:
    for line in f:
        if line.startswith("#"):
            continue
        info = line.strip().split("\t")[7]
        for entry in info.split(";"):
            if entry.startswith("CSQ="):
                csq_entries = entry[len("CSQ="):].split(",")
                for csq in csq_entries:
                    fields = csq.split("|")
                    if len(fields) > pheno_index:
                        pheno = fields[pheno_index]
                        if pheno:
                            for trait in pheno.split("&"):
                                unique_traits.add(trait)
                break

print(f"\nNumber of unique traits/disease conditions: {len(unique_traits)}")
print("traits:", list(unique_traits)[:2])


Number of unique traits/disease conditions: 2
traits: ['1', '0']


In [8]:
## List pathogenic/likely pathogenic variants
pathogenic_variants = []
with open(vcf_file) as f:
    for line in f:
        if line.startswith("#"):
            continue
        cols = line.strip().split("\t")
        chrom, pos = cols[0], cols[1]
        info = cols[7]
        for entry in info.split(";"):
            if entry.startswith("CSQ="):
                csq_entries = entry[len("CSQ="):].split(",")
                for csq in csq_entries:
                    fields = csq.split("|")
                    if len(fields) > clin_index:
                        clin_sig = fields[clin_index].lower()
                        if clin_sig in ["pathogenic", "likely_pathogenic", "likely pathogenic"]:  # only these two
                            pathogenic_variants.append(f"{chrom}:{pos} {clin_sig}")
                break

print(f"\nNumber of pathogenic/likely pathogenic variants: {len(pathogenic_variants)}")
print("Some examples:")
print("\n".join(pathogenic_variants[:20]))


Number of pathogenic/likely pathogenic variants: 16
Some examples:
1:155264120 pathogenic
1:155264120 pathogenic
1:155264120 pathogenic
1:155264120 pathogenic
1:155264120 pathogenic
1:155264120 pathogenic
1:226923505 likely_pathogenic
1:226923505 likely_pathogenic
1:226923505 likely_pathogenic
1:226923938 likely_pathogenic
1:226923938 likely_pathogenic
1:226923938 likely_pathogenic
6:38821025 pathogenic
6:38821025 pathogenic
6:38821025 pathogenic
6:38821025 pathogenic


In [9]:
## Per chromosome: count variations and SNP IDs, identify variants without SNP IDs
total_variants = defaultdict(int)
with_snpid = defaultdict(int)
without_snpid_variants = defaultdict(list)

for rec in vcf.fetch():
    chrom = rec.chrom
    total_variants[chrom] += 1
    if rec.id != ".":
        with_snpid[chrom] += 1
    else:
        without_snpid_variants[chrom].append(f"{chrom}:{rec.pos}")  # store chr:pos for variants without ID

# Print summary
print(f"{'CHROM':<6} {'TOTAL':<6} {'WITH_ID':<8} {'WITHOUT_ID':<10}")
for chrom in sorted(total_variants.keys()):
    total = total_variants[chrom]
    with_id = with_snpid.get(chrom, 0)
    without_id = total - with_id
    print(f"{chrom:<6} {total:<6} {with_id:<8} {without_id:<10}")

CHROM  TOTAL  WITH_ID  WITHOUT_ID
1      3112   3112     0         
10     1359   1359     0         
11     2100   2100     0         
12     1544   1544     0         
13     551    551      0         
14     1125   1125     0         
15     1023   1023     0         
16     1207   1207     0         
17     1872   1872     0         
18     490    490      0         
19     2226   2226     0         
2      2017   2017     0         
20     830    830      0         
21     435    435      0         
22     772    772      0         
3      1763   1763     0         
4      1269   1269     0         
5      1363   1363     0         
6      1655   1655     0         
7      1322   1322     0         
8      1010   1010     0         
9      1271   1271     0         
X      408    408      0         
Y      2      2        0         


In [10]:
## Write log file
log_file = "vcf1_analysis_log.txt"
with open(log_file, "w") as f:
    f.write(f"Total variants in '{vcf_file}': {variant_count}\n\n")
    
    f.write("=== Unique Genes ===\n")
    f.write(f"Total unique genes: {len(unique_genes)}\n")
    # Few examples of unique genes after total count
    f.write("Few examples of unique genes:\n")
    f.write(", ".join(list(unique_genes)[:20]) + "\n\n")

    f.write("=== Unique Traits/Disease Conditions ===\n")
    f.write(f"Total unique traits: {len(unique_traits)}\n")
    f.write(", ".join(list(unique_traits)[:50]) + "\n\n")

    f.write("=== Pathogenic/Likely Pathogenic Variants ===\n")
    f.write(f"Total pathogenic variants: {len(pathogenic_variants)}\n")
    f.write("\n".join(pathogenic_variants[:50]) + "\n\n")

    f.write("=== Per-Chromosome Variant Summary ===\n")
    f.write(f"{'CHROM':<6} {'TOTAL':<6} {'WITH_ID':<8} {'WITHOUT_ID':<10}\n")
    for chrom in sorted(total_variants.keys()):
        total = total_variants[chrom]
        with_id = with_snpid.get(chrom, 0)
        without_id = total - with_id
        f.write(f"{chrom:<6} {total:<6} {with_id:<8} {without_id:<10}\n")

print(f"\nComprehensive log written to {log_file}")


Comprehensive log written to vcf1_analysis_log.txt
